First we import necessary packages.

In [ ]:
import pandas as pd
import re
import random

Now we load the data and take a quick look at the first 5 rows.

In [ ]:
mydata = pd.read_excel('/content/121821_Digi_Comments_Final_RA_V2.xlsx')

In [ ]:
mydata.loc[1]

SKU Description    \n                                    دستمال ک...
Comment                                    خوب هست ولی یکم گرون هست🧻
Attitude                                                           +
Brand                                                      Easy Pick
Category                                                        Soft
Name: 1, dtype: object

We have some extra whitespace in the SKU Description column. Let's clean it.

In [ ]:
mydata['SKU Description']=mydata['SKU Description'].str.strip()

How many brands do we have?

In [ ]:
len(set(mydata['Brand']))

30

How many rows and columns do we have?

In [ ]:
print(mydata.shape)
print(mydata.columns)

(12028, 5)
Index(['SKU Description', 'Comment', 'Attitude', 'Brand', 'Category'], dtype='object')


The most important part of the data is the comments column. Lets's split words for further investigation.

In [ ]:
words=mydata['Comment'].str.split()
words[1]

['خوب', 'هست', 'ولی', 'یکم', 'گرون', 'هست🧻']

Now we check frequency of the words:

In [ ]:
words_ref=dict()
for i in range(1,len(words)):
  for j in range(1,len(words[i])):
    if words[i][j] in words_ref:
      words_ref[words[i][j]]=words_ref[words[i][j]]+1
    else:
        words_ref[words[i][j]]=1

And sort them by **frequency**

In [ ]:
w_ref_sort={k: v for k, v in sorted(words_ref.items(), key=lambda item: item[1], reverse=True)}

There are many words out there that are useless, so we need more cleaning. First we keep only words with frequency higher than 5

In [ ]:
m=len(list(w_ref_sort.items()))
f_ref=dict()
for j in range(0,m-1):
  if list(w_ref_sort.items())[j][1]>5:
    f_ref[list(w_ref_sort.items())[j][0]]=list(w_ref_sort.items())[j][1]

There are many words in our dictionary that are useless, such as 'اما'
Let's remove them:

In [ ]:
del f_ref['اما']
del f_ref['بود']
del f_ref['خوب']
del f_ref['اندازه']
del f_ref['قبول']
del f_ref['ولی']
del f_ref['خیلی']
del f_ref['دستمال']
del f_ref['این']
del f_ref['انگیز']
del f_ref['برای']

Words with length of 2 or less such as 'با' are useless. Let's remove them.

In [ ]:
f1_ref=dict()
for i in range(0,len(f_ref)-1):
  if len(list(f_ref.items())[i][0])>2:
    f1_ref[list(f_ref.items())[i][0]]=list(f_ref.items())[i][1]

In [ ]:
len(f1_ref)

1031

Now we can examine the final dictionary and decide about the main topics that are discussed in the comments.

In [ ]:
list(f1_ref.items())

[('کیفیت', 1463),
 ('خوبه', 1324),
 ('قیمت', 1274),
 ('داره', 1087),
 ('خوبی', 829),
 ('مناسب', 786),
 ('شگفت', 750),
 ('تخفیف', 652),
 ('عالی', 652),
 ('قیمتش', 628),
 ('هست', 615),
 ('نیست', 613),
 ('خرید', 608),
 ('راضی', 583),
 ('نسبت', 572),
 ('است', 546),
 ('صرفه', 522),
 ('استفاده', 513),
 ('خریدم', 422),
 ('ارزش', 399),
 ('میکنم', 393),
 ('بسته', 379),
 ('های', 356),
 ('میشه', 336),
 ('عالیه', 331),
 ('کاغذی', 323),
 ('برند', 300),
 ('کیفیتش', 298),
 ('جذب', 296),
 ('قابل', 294),
 ('شده', 286),
 ('کالا', 284),
 ('دیجی', 264),
 ('پیشنهاد', 261),
 ('دارد', 257),
 ('بودم', 255),
 ('بندی', 252),
 ('توجه', 244),
 ('دیگه', 235),
 ('نداره', 232),
 ('تنو', 228),
 ('نظر', 227),
 ('همیشه', 225),
 ('جنس', 223),
 ('معمولی', 221),
 ('فقط', 221),
 ('نازک', 220),
 ('خوبیه', 214),
 ('بالا', 211),
 ('بسیار', 207),
 ('راضیم', 206),
 ('واقعا', 204),
 ('هستم', 196),
 ('اقتصادی', 195),
 ('داشت', 189),
 ('پاپیا', 181),
 ('محصول', 181),
 ('نمیشه', 180),
 ('نبود', 178),
 ('مناسبه', 172),
 ('توی', 160)

After examining the dictionary, we've came to conclusion that there are **20** unique features that matter to the consumer and can be extracted from our data:
**1- Price | 2- Promotion | 3- VFM(Value For Money) | 4- Brand | 5- Quality | 6- Gift | 7- Absorption | 8- Thickness (Number of Layers) | 9- Size (Volume) | 10- Smell | 11- Appearance | 12- Material (Softness) | 13- Hygienic | 14- Standard | 15- Lint | 16- Color | 17- Perforation | 18- Packaging | 19- Strength | 20- Embossed Pattern (on the tissues)**

So we create a column for each and every feature, we will put 1 in that column if the comment is talking about that feature and 0 if not.

In [ ]:
mydata['Price']=0
mydata['Promotion']=0
mydata['VFM(Value For Money)']=0
mydata['Brands']=0
mydata['Quality']=0
mydata['Gift']=0
mydata['Absorption']=0
mydata['Thickness(Number of Layers)']=0
mydata['Size(Volume)']=0
mydata['Smell']=0
mydata['Appearance']=0
mydata['Material(Softness)']=0
mydata['Hygienic']=0
mydata['Standard']=0
mydata['Lint']=0
mydata['Color']=0
mydata['Perforation']=0
mydata['Packaging']=0
mydata['Strength']=0
mydata['Embossed']=0



---

1- Let's check if consumers are talking about prices.

In [ ]:
a1=re.compile(r'.*گر.ن.*')
a2=re.compile(r'.*ارز.ن.*')
a3=re.compile(r'.*قیمت.*')
a4=re.compile(r'.*اقتصاد.*')
a5=re.compile(r'.*توم.ن.*')
a6=re.compile(r'.*پول.*')
a7=re.compile(r'.*هزینه.*')
for i in range(0,len(mydata)-1):
  if a1.search(mydata.loc[i][1])!=None or a2.search(mydata.loc[i][1])!=None or a3.search(mydata.loc[i][1])!=None or a4.search(mydata.loc[i][1])!=None or a5.search(mydata.loc[i][1])!=None or a6.search(mydata.loc[i][1])!=None or a7.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Price']=1

2- promotions.

In [ ]:
b1=re.compile(r'.*تخفیف.*')
b2=re.compile(r'.*شگفت.*')
b3=re.compile(r'.*ویژه.*')
b4=re.compile(r'.*فرایدی.*')
b5=re.compile(r'.*حراج.*')
for i in range(0,len(mydata)-1):
  if b1.search(mydata.loc[i][1])!=None or b2.search(mydata.loc[i][1])!=None or b3.search(mydata.loc[i][1])!=None or b4.search(mydata.loc[i][1])!=None or b5.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Promotion']=1

3- VFM(Value For Money).

In [ ]:
c1=re.compile(r'.*صرفه.*')
c2=re.compile(r'.*صرفس.*')
c3=re.compile(r'.*ارزش.*')
c4=re.compile(r'.*میارزه.*')
c5=re.compile(r'.*راه انداز.*')
c6=re.compile(r'.*میرزه.*')
for i in range(0,len(mydata)-1):
  if c1.search(mydata.loc[i][1])!=None or c2.search(mydata.loc[i][1])!=None or c3.search(mydata.loc[i][1])!=None or c4.search(mydata.loc[i][1])!=None or c5.search(mydata.loc[i][1])!=None or c6.search(mydata.loc[i][1])!=None:
    mydata.at[i,'VFM(Value For Money)']=1

4- Brands.

In [ ]:
d1=re.compile(r'.*حریر.*')
d2=re.compile(r'.*ایزی.*')
d3=re.compile(r'.*بیتا.*')
d4=re.compile(r'.*نانسی.*')
d5=re.compile(r'.*پاپیا.*')
d6=re.compile(r'.*تنو.*')
d7=re.compile(r'.*گلریز.*')
d8=re.compile(r'.*برند.*')
d9=re.compile(r'.*سافتلن.*')
d10=re.compile(r'.*مارک.*')
d11=re.compile(r'.*چشمک.*')
d12=re.compile(r'.*مرحبا.*')
d13=re.compile(r'.*اسنیل.*')
d14=re.compile(r'.*مارك.*')
for i in range(0,len(mydata)-1):
  if d1.search(mydata.loc[i][1])!=None or d2.search(mydata.loc[i][1])!=None or d3.search(mydata.loc[i][1])!=None or d4.search(mydata.loc[i][1])!=None or d5.search(mydata.loc[i][1])!=None or d6.search(mydata.loc[i][1])!=None or d7.search(mydata.loc[i][1])!=None or d8.search(mydata.loc[i][1])!=None or d9.search(mydata.loc[i][1])!=None or d10.search(mydata.loc[i][1])!=None or d11.search(mydata.loc[i][1])!=None or d12.search(mydata.loc[i][1])!=None or d13.search(mydata.loc[i][1])!=None or d14.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Brands']=1

5- Quality.

In [ ]:
e1=re.compile(r'.*کیفیت.*')
for i in range(0,len(mydata)-1):
  if e1.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Quality']=1

6- Gift.

In [ ]:
f1=re.compile(r'.*سینی.*')
f2=re.compile(r'.*اشانت.*')
for i in range(0,len(mydata)-1):
  if f1.search(mydata.loc[i][1])!=None or f2.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Gift']=1

7- Absorption.

In [ ]:
g1=re.compile(r'.*چسب.*')
g2=re.compile(r'.*خرد.*')
g3=re.compile(r'.*رطوبت.*')
g4=re.compile(r'.*آبگیر.*')
g5=re.compile(r'.*قدرت.*')
g6=re.compile(r'.*جذب.*')
for i in range(0,len(mydata)-1):
  if g1.search(mydata.loc[i][1])!=None or g2.search(mydata.loc[i][1])!=None or g3.search(mydata.loc[i][1])!=None or g4.search(mydata.loc[i][1])!=None or g5.search(mydata.loc[i][1])!=None or g6.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Absorption']=1

8- Thickness (Number of Layers).

In [ ]:
h1=re.compile(r'.*لایه.*')
h2=re.compile(r'.*ضخ.م.*')
h3=re.compile(r'.*کلفت.*')
h4=re.compile(r'.*نازک.*')
for i in range(0,len(mydata)-1):
  if h1.search(mydata.loc[i][1])!=None or h2.search(mydata.loc[i][1])!=None or h3.search(mydata.loc[i][1])!=None or h4.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Thickness(Number of Layers)']=1

9- Size (Volume).

In [ ]:
i1=re.compile(r'.*تعداد.*')
i2=re.compile(r'.*حجم.*')
i3=re.compile(r'.*برش.*')
i4=re.compile(r'.*متراژ.*')
i5=re.compile(r'.*سایز.*')
i6=re.compile(r'.*ابعاد.*')
i7=re.compile(r'.*سانت.*')
i8=re.compile(r'.*سبک.*')
for i in range(0,len(mydata)-1):
  if i1.search(mydata.loc[i][1])!=None or i2.search(mydata.loc[i][1])!=None or i3.search(mydata.loc[i][1])!=None or i4.search(mydata.loc[i][1])!=None or i5.search(mydata.loc[i][1])!=None or i6.search(mydata.loc[i][1])!=None or i7.search(mydata.loc[i][1])!=None or i8.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Size(Volume)']=1

10- Smell.

In [ ]:
j1=re.compile(r'.*بوی.*')
for i in range(0,len(mydata)-1):
  if j1.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Smell']=1

11- Appearance.

In [ ]:
k1=re.compile(r'.*زیبا.*')
k2=re.compile(r'.*ظاهر.*')
k3=re.compile(r'.*طرح.*')
k4=re.compile(r'.*شیک.*')
k5=re.compile(r'.*قشنگ.*')
k6=re.compile(r'.*طراحی.*')
for i in range(0,len(mydata)-1):
  if k1.search(mydata.loc[i][1])!=None or k2.search(mydata.loc[i][1])!=None or k3.search(mydata.loc[i][1])!=None or k4.search(mydata.loc[i][1])!=None or k5.search(mydata.loc[i][1])!=None or k6.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Appearance']=1

12- Material (Softness).

In [ ]:
l1=re.compile(r'.*بازیافت.*')
l2=re.compile(r'.*لط.ف.*')
l3=re.compile(r'.*خمیر.*')
l4=re.compile(r'.*نرم.*')
l5=re.compile(r'.*جنس.*')
l6=re.compile(r'.*بافت.*')
l7=re.compile(r'.*مواد.*')
l8=re.compile(r'.*زبر.*')
for i in range(0,len(mydata)-1):
  if l1.search(mydata.loc[i][1])!=None or l2.search(mydata.loc[i][1])!=None or l3.search(mydata.loc[i][1])!=None or l4.search(mydata.loc[i][1])!=None or l5.search(mydata.loc[i][1])!=None or l6.search(mydata.loc[i][1])!=None or l7.search(mydata.loc[i][1])!=None or l8.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Material(Softness)']=1

13- Hygienic.

In [ ]:
m1=re.compile(r'.*لودگ.*')
m2=re.compile(r'.*بهداشت.*')
m3=re.compile(r'.*کثیف.*')
for i in range(0,len(mydata)-1):
  if m1.search(mydata.loc[i][1])!=None or m2.search(mydata.loc[i][1])!=None or m3.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Hygienic']=1

14- Standards.

In [ ]:
n1=re.compile(r'.*استاندارد.*')
for i in range(0,len(mydata)-1):
  if n1.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Standard']=1

15- Lint.

In [ ]:
o1=re.compile(r'.*گوله.*')
o2=re.compile(r'.*پرز.*')
o3=re.compile(r'.*گلوله.*')
for i in range(0,len(mydata)-1):
  if o1.search(mydata.loc[i][1])!=None or o2.search(mydata.loc[i][1])!=None or o3.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Lint']=1

16- Color.

In [ ]:
p1=re.compile(r'.*رنگ.*')
p2=re.compile(r'.*تیره.*')
for i in range(0,len(mydata)-1):
  if p1.search(mydata.loc[i][1])!=None or p2.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Color']=1

17- Perforation.

In [ ]:
q1=re.compile(r'.*پرفراژ.*')
for i in range(0,len(mydata)-1):
  if q1.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Perforation']=1

18- Packaging.

In [ ]:
r1=re.compile(r'.*بندی.*')
r2=re.compile(r'.*پارگی.*')
r3=re.compile(r'.*مچاله.*')
r4=re.compile(r'.*کارتن.*')
r5=re.compile(r'.*جعبه.*')
r6=re.compile(r'.*نایلون.*')
r7=re.compile(r'.*کاور.*')
r8=re.compile(r'.*مقوایی.*')
for i in range(0,len(mydata)-1):
  if r1.search(mydata.loc[i][1])!=None or r2.search(mydata.loc[i][1])!=None or r3.search(mydata.loc[i][1])!=None or r4.search(mydata.loc[i][1])!=None or r5.search(mydata.loc[i][1])!=None or r6.search(mydata.loc[i][1])!=None or r7.search(mydata.loc[i][1])!=None or r8.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Packaging']=1

19- Strength.

In [ ]:
s1=re.compile(r'.*محکم.*')
s2=re.compile(r'.*مقاوم.*')
s3=re.compile(r'.*پود.*')
s4=re.compile(r'.*متلاشی.*')
s5=re.compile(r'.*استحکام.*')
for i in range(0,len(mydata)-1):
  if s1.search(mydata.loc[i][1])!=None or s2.search(mydata.loc[i][1])!=None or s3.search(mydata.loc[i][1])!=None or s4.search(mydata.loc[i][1])!=None or s5.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Strength']=1

20- Embossed Pattern (on the tissues).

In [ ]:
t1=re.compile(r'.*برجسته.*')
for i in range(0,len(mydata)-1):
  if t1.search(mydata.loc[i][1])!=None:
    mydata.at[i,'Embossed']=1



---



Let's see a random comment:

In [ ]:
mydata.loc[random.randint(0,len(mydata))]

SKU Description                دستمال کاغذی 150 برگ ایزی پیک طرح گل بسته 8 عددی
Comment                             اولین باره ازین برند خریدم ولی خیلی خوب بود
Attitude                                                                      +
Brand                                                                 Easy Pick
Category                                                                   Soft
Price                                                                         0
Promotion                                                                     0
VFM(Value For Money)                                                          0
Brands                                                                        1
Quality                                                                       0
Gift                                                                          0
Absorption                                                                    0
Thickness(Number of Layers)             

Below is a brief summary of the results of this analysis.

In [ ]:
print('\033[1m' + 'Total number of comments: ',mydata.shape[0],'\n')
for i in range(5,mydata.shape[1]):
  print(round(sum(mydata[mydata.columns[i]])/len(mydata)*100,2),'%',' of consumers have talked about ',mydata.columns[i],' meaning ',sum(mydata[mydata.columns[i]]),' comments.\n',sep='')

Total number of comments:  12028 

24.82% of consumers have talked about Price meaning 2985 comments.

13.29% of consumers have talked about Promotion meaning 1598 comments.

10.72% of consumers have talked about VFM(Value For Money) meaning 1289 comments.

10.52% of consumers have talked about Brands meaning 1265 comments.

27.44% of consumers have talked about Quality meaning 3301 comments.

0.58% of consumers have talked about Gift meaning 70 comments.

5.37% of consumers have talked about Absorption meaning 646 comments.

6.66% of consumers have talked about Thickness(Number of Layers) meaning 801 comments.

4.44% of consumers have talked about Size(Volume) meaning 534 comments.

0.22% of consumers have talked about Smell meaning 26 comments.

1.9% of consumers have talked about Appearance meaning 229 comments.

10.16% of consumers have talked about Material(Softness) meaning 1222 comments.

1.57% of consumers have talked about Hygienic meaning 189 comments.

0.22% of consumers hav

Finally we can export the Excel file for further examination.

In [ ]:
pd.DataFrame.to_excel(mydata,excel_writer='/content/Digi_Comments_Output.xlsx')



---



*These extra steps are for data test and manipulation, so they can be used for next versions of input data.*

In [ ]:
for i in range(0,len(mydata)-1):
  if re.search(r'.کاور.*',mydata.loc[i][1])!=None:
    print(mydata.loc[i][1])

واقعا تو دستشویی دستمال باید تو کاور پوشیده باشه 
این گزینه خوبی هست برای دستشویی هایی که کاور کامل ندارن
بدلیل داشتن کاور در سرویس بهداشتی قابل استفاده است
کیفیت دستمال خوبه و با کاوری که دارد، از آلودگی و خیس شدن، حفظ میشه
از نظر ضخامت کاغذ بدنیست ولی بسته بندیش رضایت بخش نبود چون ۱۲ تا داخل یک کاور مشمبایی قرار داشت که چندقسمت اون هم پاره بود و امکان الوده شدن دستمال وجود داشت که برای استفاده بهداشتی مطلوب نیست
کیفیت پاپیا کلا خوب است ولی بسته بندی ارسال توسط دیجی کالا کاملا غیرحرفه‌ای ای و غیر بهداشتی بود، به نحوی که مجبور  شدیم دستمال ها را از بسته بندی خارج و در کاور پلاستیکی نگهداری کنیم
همیشه از سافتلن استفاده میکنم به دلیل کیفیت خوب دستمالهایش. این بار اما از معدود دفعاتی بود که بسته ۱۲تایی گرفتم و با کاور پاره دریافتش کردم. کاش برای رعایت بهداشت در تحویل کالای بهداشتی و حساسی مثل این محصول فکری بشود. فرایند مرجوع کردن کالا هم که مستلزم تماس مکرر و کلی برنامه‌ریزی بود. این همه دردسر برای خریدن دستمال کاغذی واقعا زیادی است.


In [ ]:
print(mydata[mydata['Material(Softness)']==1]['Comment'])

3                                   خوب بود جنس خوبی داشت.
11                             نسبت به قیمتش جنس خوبی داره
12       در شگفت انگیز خریداری شد بسیار عالی و باتوجه ب...
18       توی شگفت انگیز با قیمت خوب گرفتم. ایزی پیک دست...
42                    یه مقدار کوچیکه نرم و لطیف با طرح گل
                               ...                        
11973    خوب و لطیفه و زود پودر نمیشه ، در پیشنهاد شگفت...
11991    نسبت به قیمت عالی و اینکه بعضی گفته بودن پودر ...
12017             جنس معمولی ودر شگفت انگیز ارزش خرید داره
12018               جنس خویی داره نسبت به قیمتش قابل قبوله
12025                          خیلی خوب بود و لطیف و ملایم
Name: Comment, Length: 1222, dtype: object
